In [1]:
source("~/wisdom/r/data_analysis_environment.R")


Attaching package: ‘dplyr’

The following objects are masked from ‘package:data.table’:

    between, last

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘magrittr’

The following object is masked from ‘package:tidyr’:

    extract



In [38]:
# Supplementary File 1 
read_tsv("../data/pancancer/candidateList_full.tsv") %>%
    filter(Reliable==1 & Origin=="Tumor") %>%
    select(-Reliable, -Origin) %>%
    write_tsv("../results/supplementary_files/supplementary_file_1.tsv")

ERROR: Error in eval(expr, envir, enclos): Failed to open '../results/supplementary_files/supplementary_file_1.tsv'.


In [ ]:
# Supplementary File 2

# domains

In [37]:
# Supplementary File 3
switches.split <- read_tsv("../data/pancancer/candidateList_full.tumorSplit.tsv") %>%
    filter(Reliable==1 & Origin=="Tumor") %>%
    select(Tumor,GeneId,Symbol,Normal_transcript,Tumor_transcript)

proteome <- read_tsv("../data/mutations/proteome_information.txt") %>%
    select(Tumor,Transcript)

pfams <- read_tsv("../data/structural_analysis/interpro_analysis.tsv") %>%
    select(GeneId,Symbol,Normal_transcript,Tumor_transcript,Feature,What,normalReps,tumorReps) %>%
    unique %>%
    rename(Normal_isoform_order = normalReps, Tumor_isoform_order = tumorReps, Observation = What) %>%
    mutate(Observation = ifelse(Observation == "Nothing", "No_change", Observation)) %>%
    separate(Feature, c("Pfam_id", "Name"), "\\|")

ppi.file <- "../data/eporta/raw_tables/Switched_interactions_consensus.txt"

## get max number of columns (necessary for reading)
no_col <- max(count.fields(ppi.file,sep = "\t"))
no_col.ppi <- (no_col-6)/2
ppi.cols <- paste(c("Origin","Interaction"), floor(seq(1,no_col.ppi,0.5)), sep="_")

## read table
ppi.split <- read.table(ppi.file,header=F,fill=T,col.names=1:no_col) %>%
    set_colnames(c("GeneId","Symbol","Normal_transcript","Tumor_transcript",
                   "GeneId_partner","Symbol_partner",ppi.cols)) %>%
    # all Origin columns contail "DDI_match", so we can disregard them
    select(-starts_with("Origin_")) %>%
    # convert from wide to long table format
    reshape2::melt(id.vars = c("GeneId","Symbol","Normal_transcript","Tumor_transcript",
                               "GeneId_partner","Symbol_partner"), value.name = "Interaction") %>%
    select(-variable) %>%
    # remove cases with no interaction described
    filter(Interaction != "") %>%
    # split interaction information
    separate(Interaction, into = c("Effect_on_interaction","Transcript_partner","Domains"), sep = "-") %>%
    mutate(Domains = strsplit(Domains, "_")) %>%
    unnest(Domains) %>%
    separate(Domains, into = c("Pfam_id","Pfam_id_partner"), sep = "/") %>%
    mutate(Effect_on_interaction = plyr::revalue(Effect_on_interaction, 
                                           replace = c("Gained"="Gain", "Lost"="Loss", "Kept"="Unaffected"))) %>%
    merge(switches.split) %>%
    merge(proteome, by.x = c("Tumor","Transcript_partner"), by.y = c("Tumor","Transcript"))

merge(pfams, ppi.split, all = TRUE) %>%
    select(Tumor,GeneId:Tumor_isoform_order,GeneId_partner,Symbol_partner,
           Transcript_partner,Pfam_id_partner,Effect_on_interaction) %>%
    arrange(Tumor) %>%
    write_tsv("../results/supplementary_files/supplementary_file_3.tsv")

Warning message:
: attributes are not identical across measure variables; they will be dropped

In [ ]:
# Supplementary File 4

me <- lapply(1:10, function(i){
    read_tsv(paste0("../data/mutations/pannegative_mutual_exclusion.top_",i,"_drivers.txt")) %>%
        mutate(NumDrivers = i)
}) %>% 
    do.call("rbind",.) %>%
    by(., paste0(.$GeneId,.$Tumor), 
         function(X) X[which.min(X$p.me),]) %>%
    do.call("rbind",.)  %>%
    select(Tumor,GeneId,Symbol,Normal_transcript,Tumor_transcript,p.me,NumDrivers) %>%
    set_colnames(c("Tumor","GeneId","Symbol","Normal_transcript","Tumor_transcript","p_pannegative","Mutually_exclusive_drivers"))

me.specificDrivers <- read_tsv("../data/mutations/mutual_exclusion_top_drivers.txt") %>%
    filter(!is.na(Pathway))

head(me.specificDrivers)

# merge(me, me.specificDrivers, all = TRUE) %>%
#     write_tsv("../results/supplementary_files/supplementary_file_4.tsv")

# me w/ pancancer and specific gene in the same pathway